# library

In [1]:
import sys
import cv2
import numpy as np

print('version of python:',sys.version)
print('version of cv2:',cv2.__version__)
print('version of np:',np.__version__)

version of python: 3.6.10 |Anaconda, Inc.| (default, May  8 2020, 04:03:53) [MSC v.1916 32 bit (Intel)]
version of cv2: 3.4.1
version of np: 1.17.0


# pre processing

In [2]:
# src -> gray -> blur -> Canny -> closing operation
def pre_process(img,mask=(5,5)):
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    blur=cv2.GaussianBlur(gray,mask,0)
    edge=cv2.Canny(blur,100,200)
    
    dilated=cv2.dilate(edge,mask,iterations=3)
    eroded=cv2.erode(dilated,mask,iterations=3)
    
    return eroded

# common function

In [5]:
# get vertices of A5 paper
def getVertices(contours):
    cnts=sorted(contours,key=cv2.contourArea,reverse=True)[:5]
    for c in cnts:
        peri=cv2.arcLength(c,True)
        vertices=cv2.approxPolyDP(c,0.02*peri,True)
        if len(vertices)==4:
            break
    pts=vertices.reshape(4,2)
    return pts


# perspecive transform
def perspective(frame,pts):
    sm=pts.sum(axis=1)
    diff=np.diff(pts,axis=1)
    
    topLeft=pts[np.argmin(sm)]
    bottomRight=pts[np.argmax(sm)]
    topRight=pts[np.argmin(diff)]
    bottomLeft=pts[np.argmax(diff)]
    
    pts1=np.float32([topLeft,topRight,bottomRight,bottomLeft])
    
    w1=abs(bottomRight[0]-bottomLeft[0])
    w2=abs(topRight[0]-topLeft[0])
    h1=abs(topRight[1]-bottomRight[1])
    h2=abs(topLeft[1]-bottomLeft[1])
    width=max([w1,w2])
    height=max([h1,h2])
    
    pts2=np.float32([[0,0],[width-1,0],[width-1,height-1],[0,height-1]])
    
    matrix=cv2.getPerspectiveTransform(pts1,pts2)
    result=cv2.warpPerspective(frame,matrix,(width,height))
    return result,width,height


# get distance width and height
def getDist(compacted,obj_vertices,A5_W=148,A5_H=210,pad=10):
    whole_w,whole_h=compacted.shape[1]+2*pad,compacted.shape[0]+2*pad
    
    # a==mm per pixel(w), b==mm per pixel(h)
    a=A5_W/whole_w
    b=A5_H/whole_h
    
    ver=obj_vertices.copy().astype(np.int64)
    
    x=abs(ver[0][0]-ver[1][0])*a
    y=abs(ver[0][1]-ver[1][1])*b
    len1=(x**2+y**2)**0.5
    
    x=abs(ver[0][0]-ver[-1][0])*a
    y=abs(ver[0][1]-ver[-1][1])*b
    len2=(x**2+y**2)**0.5
    
    return (len1,len2) if len1>len2 else (len2,len1)

# measure object size in a Image

In [6]:
# A5 size(mm)
A5_W=148
A5_H=210

cap=cv2.VideoCapture(0)
if cap.isOpened():
    while True:
        ret, frame=cap.read()
        if ret:
            cv2.imshow('original',frame)
            ### pre process ###
            pre=pre_process(frame)
            cv2.imshow('pre',pre)

            ### perspective transform ###
            dt,contours,hierarchy=cv2.findContours(pre.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
            vertices=getVertices(contours)
            #print(vertices)
            transformed,w,h=perspective(pre,vertices)
            #print(transformed.shape)
            cv2.imshow('transformed',transformed)


            ### get roi == remove external border ###
            pad=10
            compacted=transformed[pad:transformed.shape[0]-pad,pad:transformed.shape[1]-pad]
            #cv2.imshow('compacted',compacted)
            #print(compacted.shape)


            ### get border of the object ###
            dt2,contours2,hierarchy=cv2.findContours(compacted.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
            rotateRect=cv2.minAreaRect(contours2[0])
            obj_vertices=cv2.boxPoints(rotateRect)
            #print(obj_vertices)
            #obj_vertices=np.round(obj_vertices).astype(np.uint64)
            obj_vertices=obj_vertices.astype(np.uint64)
            #print(obj_vertices)


            ### draw border of the object ###
            color=cv2.cvtColor(compacted,cv2.COLOR_GRAY2BGR)
            #cv2.drawContours(color,contours2,-1,(0,0,255),2)
            for i in range(4):
                cv2.line(color,tuple(obj_vertices[i]),tuple(obj_vertices[(i+1)%4]),(0,0,255),1)


            ### get length of the object ###
            obj_h,obj_w=getDist(compacted,obj_vertices)
            #print('length_object:',obj_w,obj_h)
            obj_h=str('h:%.2fmm'%np.round(obj_h,2))
            obj_w=str('w:%.2fmm'%np.round(obj_w,2))
            cv2.putText(color,obj_h,(obj_vertices[0][0],obj_vertices[0][1]),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,0),1)
            cv2.putText(color,obj_w,(obj_vertices[3][0],obj_vertices[3][1]),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,0),1)

            cv2.imshow('result',color)
            if cv2.waitKey(1)!=-1:
                break

cap.release()
cv2.destroyAllWindows()

ValueError: cannot reshape array of size 20 into shape (4,2)

# error

In [1]:
# ground truth 24, 31.5 (mm)
GT=np.array(sorted([31.5,24.0]))
measure=np.array(sorted(getDist(compacted,obj_vertices)))
print('Groud-Truth(mm):',GT)
print('measure(mm):',measure)
print('len error(mm):',abs(GT-measure))
print('area error(mm2):',abs(GT[0]*GT[1]-measure[0]*measure[1]))

NameError: name 'np' is not defined